In [40]:
import pandas as pd
import numpy as np
import copy
import sys
import os
sys.path.append("../")
from parser.utils import load_json, dfs_cardinality, estimate_scan_in_mb, dumper
from models.feature.single_xgboost_feature import find_top_k_operators, featurize_one_plan, get_top_k_table_by_size
from utils.load_brad_trace import load_trace, create_concurrency_dataset, load_trace_all_version
from models.concurrency.utils import pre_info_train_test_seperation
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from models.single.stage import SingleStage
from models.concurrency.complex_models import ConcurrentRNN
np.set_printoptions(suppress=True)

In [41]:
#parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_aurora/aurora_mixed_parsed_queries.json"
parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_postgres/postgres_mixed_parsed_queries.json"
plans = load_json(parsed_queries_path, namespace=False)
folder_name = "mixed_postgres"
directory = f"/Users/ziniuw/Desktop/research/Data/AWS_trace/{folder_name}/"
all_raw_trace, all_trace = load_trace_all_version(directory, 8, concat=True)
all_concurrency_df = []
for trace in all_trace:
    concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=200)
    all_concurrency_df.append(concurrency_df)
concurrency_df = pd.concat(all_concurrency_df, ignore_index=True)

In [42]:
train_trace_df_sep, eval_trace_df_sep = pre_info_train_test_seperation(concurrency_df)
print(len(train_trace_df_sep), len(eval_trace_df_sep))
np.random.seed(0)
train_idx = np.random.choice(len(concurrency_df), size=int(0.8 * len(concurrency_df)), replace=False)
test_idx = [i for i in range(len(concurrency_df)) if i not in train_idx]
train_trace_df = copy.deepcopy(concurrency_df.iloc[train_idx])
eval_trace_df = concurrency_df.iloc[test_idx]
eval_trace_df = copy.deepcopy(eval_trace_df[eval_trace_df['num_concurrent_queries'] > 0])
print(len(train_trace_df), len(eval_trace_df))

17603 13399
24889 6198


In [43]:
ss = SingleStage(use_table_features=True, true_card=True)
#df = ss.featurize_data(train_trace_df, parsed_queries_path)
df = ss.featurize_data(concurrency_df, parsed_queries_path)
ss.train(df)

Top 20 operators contains 0.9642989959092599 total operators


In [44]:
rnn = ConcurrentRNN(ss, 
                    "postgres",
                    input_size=len(ss.all_feature[0]) * 2 + 7,
                    embedding_dim=128,
                    hidden_size=256,
                    num_layers=2,
                    loss_function="q_loss",
                    last_output=True,
                    use_separation=False
                   )
rnn.train(train_trace_df, eval_trace_df, lr=0.001, loss_function="q_loss", val_on_test=True)
#rnn.load_model("checkpoints")

********Epoch 0, training loss: 411.0203755201438 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.99it/s]


50% absolute error is 3.804101347923279, q-error is 1.6581772565841675
90% absolute error is 55.93377799987793, q-error is 4.753993654251099
95% absolute error is 103.91797485351515, q-error is 7.097725009918208
********Epoch 5, training loss: 9.051576638756654 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 17.02it/s]


50% absolute error is 2.7063283920288086, q-error is 1.352875530719757
90% absolute error is 22.655736351013186, q-error is 3.2635942935943607
95% absolute error is 39.13712730407712, q-error is 4.99490792751311
********Epoch 10, training loss: 0.2664264956345925 || evaluation loss: ********


...

100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.45it/s]


50% absolute error is 2.470977306365967, q-error is 1.2888217568397522
90% absolute error is 19.95462989807129, q-error is 2.8704358339309692
95% absolute error is 34.173838806152304, q-error is 4.312779402732845
********Epoch 50, training loss: 0.14900413591128128 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.51it/s]


50% absolute error is 2.673985719680786, q-error is 1.2745469212532043
90% absolute error is 19.149652099609384, q-error is 2.8148646116256715
95% absolute error is 28.345897293090815, q-error is 4.04917025566101


KeyboardInterrupt: 

In [60]:
rnn.train(train_trace_df, eval_trace_df, lr=0.001, loss_function="q_loss", val_on_test=True)

********Epoch 0, training loss: 11.324541861353776 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.57it/s]


50% absolute error is 2.5366926193237305, q-error is 1.2759972214698792
90% absolute error is 19.599263000488282, q-error is 2.876849460601807
95% absolute error is 33.53456020355224, q-error is 4.297272634506223
********Epoch 5, training loss: 8.945446717204192 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.34it/s]


50% absolute error is 2.732142925262451, q-error is 1.2789133787155151
90% absolute error is 19.87552127838136, q-error is 2.9121077775955206
95% absolute error is 31.177502822875933, q-error is 4.191343188285827
********Epoch 10, training loss: 16.398986326998624 || evaluation loss: ********


...

50% absolute error is 2.391947388648987, q-error is 1.2602429389953613
90% absolute error is 17.439052963256838, q-error is 2.723058104515076
95% absolute error is 27.66320533752439, q-error is 4.170677065849303
********Epoch 70, training loss: 15.436853009118483 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.80it/s]


50% absolute error is 2.5813673734664917, q-error is 1.2644597291946411
90% absolute error is 19.44078750610352, q-error is 2.6853895187377943
95% absolute error is 29.514906120300274, q-error is 3.995923268795012


[E thread_pool.cpp:130] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:130] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

In [53]:
rnn.save_model("checkpoints")

In [57]:
rnn2 = ConcurrentRNN(ss, 
                     "postgres",
                    input_size=len(ss.all_feature[0]) * 2 + 7,
                    embedding_dim=128,
                    hidden_size=256,
                    num_layers=2,
                    loss_function="q_loss",
                    last_output=True,
                    use_separation=False
                   )
rnn2.load_model("checkpoints")

In [54]:
preds, labels = rnn.predict(eval_trace_df, use_pre_info_only=False)

100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.61it/s]

50% absolute error is 2.5743322372436523, q-error is 1.2749279141426086
90% absolute error is 18.791236114501956, q-error is 2.882374620437622
95% absolute error is 28.05649528503415, q-error is 4.55170130729675


In [61]:
concurrency_df = create_concurrency_dataset(all_trace[0], engine=None, pre_exec_interval=200)
preds, labels = rnn2.predict(concurrency_df, use_pre_info_only=False)

100%|███████████████████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 23.03it/s]

50% absolute error is 1.6386100053787231, q-error is 1.2225273251533508
90% absolute error is 14.885701751708984, q-error is 2.4476436853408807
95% absolute error is 23.693497276306143, q-error is 3.6519856929779038


In [58]:
preds2, labels2 = rnn2.predict(eval_trace_df, use_pre_info_only=False)

100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.33it/s]

50% absolute error is 2.5743322372436523, q-error is 1.2749279141426086
90% absolute error is 18.791236114501956, q-error is 2.882374620437622
95% absolute error is 28.05649528503415, q-error is 4.55170130729675


In [48]:
i = 210
idx = np.argsort(preds2[i])
print(len(idx))
np.stack((np.asarray(preds2[i])[idx], np.asarray(preds[i])[idx], np.asarray(labels2[i])[idx]), axis=1)

111


array([[  2.8291447,   7.1109896,  79.540054 ],
       [  2.8653407,   2.3412442,   3.416112 ],
       [  2.934177 ,   4.0760264,   4.0271263],
       [  2.9910045,   3.7841144,   3.8717294],
       [  3.0511217,   5.0034904,   3.1730235],
       [  3.148857 ,  18.968218 ,   3.1449356],
       [  3.170916 ,   6.0808353, 106.32712  ],
       [  3.183551 ,   6.5285897,   4.7487164],
       [  3.2096725,  33.221123 ,   2.8220966],
       [  3.2253332,   5.26631  ,   3.3115742],
       [  3.246746 ,   4.4229527,   5.8701944],
       [  3.2892797,   5.2571554,   7.374345 ],
       [  3.2927618,  24.601357 ,   2.9005537],
       [  3.3001895,   3.752159 ,   4.9782295],
       [  3.3236983,   3.4375095,   4.804597 ],
       [  3.3453178,   4.3794765,   3.0326128],
       [  3.4195132,   5.3936315,   8.1688   ],
       [  3.4844189,   6.0924673,   4.3422008],
       [  3.5340056,   4.0386086,   4.554105 ],
       [  3.5757537,   8.585522 ,  32.302185 ],
       [  3.5846162,   6.08425  ,   5.50

In [8]:
i = 210
idx = np.argsort(preds2[i])
print(len(idx))
np.stack((np.asarray(preds[i])[idx], np.asarray(preds2[i])[idx], np.asarray(labels2[i])[idx]), axis=1)

NameError: name 'preds2' is not defined

In [11]:
i = 2
idx = np.argsort(preds[i])
print(len(idx))
np.stack((np.asarray(preds[i])[idx], np.asarray(labels[i])[idx]), axis=1)

7


array([[  9.086283,  10.558472],
       [  9.326768,  10.257236],
       [ 16.062849,  39.89623 ],
       [ 20.419281,  69.41913 ],
       [ 35.720085,  52.07332 ],
       [ 44.310814, 110.97227 ],
       [ 50.8906  ,  45.67449 ]], dtype=float32)

In [11]:
with open(directory + "postgres_mixed.sql", "r") as f:
    sql = f.readlines()

In [13]:
plans = load_json(parsed_queries_path, namespace=False)

In [16]:
aurora_sql = plans["sql_queries"]

In [15]:
plans.keys()

dict_keys(['parsed_plans', 'parsed_queries', 'sql_queries', 'database_stats', 'run_kwargs', 'skipped', 'blocks_accessed'])

In [21]:
all_index = []
for q in sql:
    i = aurora_sql.index(q.strip())
    all_index.append(i)

In [22]:
all_index

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186

In [23]:
sql[-1]

'SELECT AVG(imdb_id), AVG(kind_id), AVG(production_year) FROM title WHERE id < 42000000;'

In [25]:
sql[206]

'SELECT AVG(person_id), AVG(movie_id) FROM cast_info WHERE id < 25000000;\n'

In [26]:
aurora_sql[207]

'SELECT AVG(imdb_id), AVG(kind_id), AVG(production_year) FROM title WHERE id < 42000000;'

In [28]:
aurora_sql[-5:]

['SELECT AVG(movie_id), AVG(info_type_id) FROM movie_info WHERE id < 32000000;',
 'SELECT AVG(movie_id), AVG(keyword_id) FROM movie_keyword WHERE id < 42000000;',
 'SELECT AVG(imdb_id) FROM name WHERE id < 42000000;',
 'SELECT AVG(person_id) FROM person_info WHERE id < 42000000;',
 'SELECT AVG(imdb_id), AVG(kind_id), AVG(production_year) FROM title WHERE id < 42000000;']

In [29]:
postgres_plans = copy.deepcopy(plans)

In [30]:
postgres_plans['parsed_plans'] = [plans['parsed_plans'][i] for i in all_index]
postgres_plans['parsed_queries'] = [plans['parsed_queries'][i] for i in all_index]
postgres_plans['sql_queries'] = sql


In [39]:
import json
with open("/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_postgres/postgres_mixed_parsed_queries.json", "w") as f:
    json.dump(postgres_plans, f)